# Q2

In [105]:
import numpy as np
import pandas as pd
from numpy import log2 as log
from IPython.display import display

### Read Data.

In [106]:
data_train = pd.read_csv('prison_dataset.csv')
display(data_train.shape)
display(data_train)

(15424, 11)

,Fiscal Year Released,Recidivism Reporting Year,Race - Ethnicity,Age At Release,Convicting Offense Classification,Convicting Offense Type,Convicting Offense Subtype,Main Supervising District,Release Type,Part of Target Population,Recidivism - Return to Prison numeric
0,2010,2013,White,<45,D Felony,Violent,Other,3JD,Parole,Yes,1
1,2010,2013,White,>45,D Felony,Other,Other,3JD,Parole,Yes,1
2,2010,2013,White,<45,D Felony,Other,Other,5JD,Parole,Yes,1
3,2010,2013,White,>45,Other Felony,Drug,Trafficking,3JD,Parole,Yes,1
4,2010,2013,Black,<45,D Felony,Drug,Trafficking,3JD,Parole,Yes,1
...,...,...,...,...,...,...,...,...,...,...,...
15419,2015,2018,White,<45,Other Felony,Violent,Other,3JD,Discharged End of Sentence,Yes,0
15420,2015,2018,White,<45,D Felony,Other,Other,5JD,Discharged End of Sentence,No,0
15421,2015,2018,Black,<45,Other Felony,Violent,Other,3JD,Discharged End of Sentence,Yes,0
15422,2015,2018,White,<45,D Felony,Drug,Other,5JD,Parole,No,0


# a. ‫‪ID3‬‬

In [107]:
def cross_validation(df,per):
    data = df.sample(frac=1)
    percent = int(data.shape[0]*per/100)
    X_train = data.iloc[0:percent,:]
    X_test = data.iloc[percent:-1,:]
    return X_train,X_test
    


def Entropy(S):
    label = S.iloc[:,-1].value_counts()
    p = label/label.sum()
    Entropy = (-p*np.log2(p)).sum()
    
    return Entropy
    
def Information_Gain(S,A):
    entro_s = Entropy(S)
    Info_G = {}
    
    for attri in A:
        entro_attri = 0
        features = S[attri].unique()
        total_S_num = S[attri].shape[0]
        
        for value in features:
            value_S_num = S[S[attri] == value].shape[0]
            entro_attri += (value_S_num/total_S_num)*Entropy(S[S[attri] == value])
        Info_G[attri]= entro_s - entro_attri    
        
    return max(Info_G, key=Info_G.get)
    

def ID3(df,attributes,depth,tree=None):
    
    #remove target-label
    target = 'Recidivism - Return to Prison numeric'
    
    #choose best root!
    root = Information_Gain(df,attributes)
    if tree is None:
        tree = {}
        tree[root] = {}
    #depth of tree
    depth += 1
    #remove decision attribute
    attributes.remove(root)
    
    branches = df[root].unique().tolist()
    for child in branches:
        if(depth == 3):
            label = df[df[root]==child][target].value_counts().idxmax()
            tree[root][child] = label
            
        elif(len(attributes)==0):
            label = df[df[root]==child][target].value_counts().idxmax()
            tree[root][child] = label
            
        elif(len(df[df[root]==child][target].unique())==1):
            label = df[df[root]==child][target].value_counts().idxmax()
            tree[root][child] = label
            

        else:
            tree[root][child] =ID3(df[df[root]==child],attributes,depth)
    
    return tree

def main(data):
    #split data
    X_train, X_test = cross_validation(data,80)
    
    #depth of tree
    depth = 0
    #get all attributes of data
    attributes = data.columns.tolist()
    
    #remove target attribute
    attributes.remove('Recidivism - Return to Prison numeric')
    
    
    #call decision tree!
    result = ID3(X_train,attributes,depth)
    return result, X_test


# Tree:

In [108]:
ID3_Tree , X_test = main(data_train)
ID3_Tree

{'Fiscal Year Released': {2015: {'Release Type': {'Parole': {'Convicting Offense Type': {'Violent': 0,
      'Drug': 0,
      'Other': 0}},
    'Discharged End of Sentence': {'Main Supervising District': {'3JD': 0,
      '5JD': 0}}}},
  2010: 1,
  2013: {'Age At Release': {'<45': {'Convicting Offense Classification': {'Other Felony': 1,
      'D Felony': 1}},
    '>45': {'Race - Ethnicity': {'Black': 1, 'White': 1}}}}}}

# Test-Data Prediction:

In [109]:
def dfs_id3_classify(id3,node,df,columns,X_test):
    if(type(id3[node]) == dict):
        for key in id3[node].keys():
            data = df.copy()
            if node in columns:
                data = data[data[node] == key]
            dfs_id3_classify(id3[node],key,data,columns,X_test)
    else:
        X_test.loc[df.index.tolist(),'Y_predict'] = id3[node]
        return
    return X_test


X_test['Y_predict'] = -1
columns = X_test.columns.tolist()
X_Test = dfs_id3_classify(ID3_Tree,list(ID3_Tree.keys())[0],X_test,columns,X_test)
X_Test

,Fiscal Year Released,Recidivism Reporting Year,Race - Ethnicity,Age At Release,Convicting Offense Classification,Convicting Offense Type,Convicting Offense Subtype,Main Supervising District,Release Type,Part of Target Population,Recidivism - Return to Prison numeric,Y_predict
8946,2015,2018,Black,>45,Other Felony,Drug,Trafficking,5JD,Parole,No,0,0
13208,2015,2018,Black,<45,Other Felony,Other,Other,5JD,Discharged End of Sentence,Yes,0,0
803,2010,2013,White,<45,D Felony,Drug,Trafficking,3JD,Discharged End of Sentence,No,1,1
12058,2015,2018,White,<45,D Felony,Other,Other,3JD,Parole,Yes,1,0
8440,2015,2018,Black,<45,D Felony,Other,Other,3JD,Parole,No,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8847,2015,2018,White,<45,Other Felony,Drug,Trafficking,5JD,Parole,No,0,0
7403,2015,2018,White,<45,Other Felony,Drug,Trafficking,5JD,Parole,No,1,0
1143,2010,2013,White,<45,D Felony,Other,Other,3JD,Parole,Yes,1,1
2055,2010,2013,White,<45,D Felony,Violent,Other,3JD,Discharged End of Sentence,No,1,1


# Confusion_Matrix:

In [110]:
def Confusion_Matrix(X_Test):
    cnt_TP = 0
    cnt_TN = 0
    cnt_FP = 0
    cnt_FN = 0
    for row in range(X_Test.shape[0]):
        if(X_Test.iloc[row,-1] == X_Test.iloc[row,-2]==1):
            cnt_TP +=1
        if(X_Test.iloc[row,-1] == X_Test.iloc[row,-2]==0):
            cnt_TN +=1
        if(X_Test.iloc[row,-1] == 1 and X_Test.iloc[row,-2]==0):
            cnt_FN +=1
        if(X_Test.iloc[row,-1] == 0 and X_Test.iloc[row,-2]==1):
            cnt_FP +=1
    cm = np.array([cnt_TP, cnt_FN, cnt_FP, cnt_TN]).reshape(2,2)
    acc = (cnt_TP+cnt_TN)/X_Test.shape[0]*100
    pres = cnt_TP/(cnt_TP+cnt_FP)*100
    recall = cnt_TP/(cnt_TP+cnt_FN)*100
    F1 = 2*pres*recall/(pres+recall)
    classes = ['Positive','Negative']
    total_classes = len(classes)
    level_labels = [total_classes*[0], list(range(total_classes))]
    cm_frame1 = pd.DataFrame(data=cm,columns=pd.MultiIndex(levels=[['Predicted:'],\
                                                                  classes],codes=level_labels),\
                        index=pd.MultiIndex(levels=[['Actual:'], classes], codes=level_labels))
    criteria=pd.DataFrame(data={'Accuracy':acc,'Precision':pres,'Recall':recall,'F1_Score':F1},index=['0'])
    return cm_frame1 ,criteria
    
    
    

# Results for ID3:

In [111]:
Cm1, Cm2 = Confusion_Matrix(X_Test)  
display(Cm1)
display(Cm2)

Predicted:         
                   Positive Negative
Actual: Positive       1058      204
        Negative        667     1155

,Accuracy,Precision,Recall,F1_Score
0,71.757458,61.333333,83.835182,70.840308


# b. Random Forest

In [112]:
import random

def Entropy(S):
    label = S.iloc[:,-1].value_counts()
    p = label/label.sum()
    Entropy = (-p*np.log2(p)).sum()
    
    return Entropy

def Information_Gain_forest(S,A,m_feature):
    entro_s = Entropy(S)
    Info_G = {}
    Info_G.clear()
    m_Attri = []
    if(m_feature <= len(A)):
        m_Attri=random.sample(A,m_feature)    
    elif(m_feature > len(A)):
        m_Attri = A
        
    for attri in m_Attri:
        entro_attri = 0
        features = S[attri].unique()
        total_S_num = S[attri].shape[0]
        
        for value in features:
            value_S_num = S[S[attri] == value].shape[0]
            entro_attri += (value_S_num/total_S_num)*Entropy(S[S[attri] == value])
        Info_G[attri]= entro_s - entro_attri    
        
    return max(Info_G, key=Info_G.get)

def ID3_forest(df,attributes,depth,m_feature,tree=None):
    
    #remove target-label
    target = 'Recidivism - Return to Prison numeric'
    
    #choose best root!
    root = Information_Gain_forest(df,attributes,m_feature)
    if tree is None:
        tree = {}
        tree[root] = {}
    #depth of tree
    depth += 1
    #remove decision attribute
    attributes.remove(root)
    
    branches = df[root].unique().tolist()
    for child in branches:
        if(depth == 3):
            label = df[df[root]==child][target].value_counts().idxmax()
            tree[root][child] = label
            
        elif(len(attributes)==0):
            label = df[df[root]==child][target].value_counts().idxmax()
            tree[root][child] = label
            
        elif(len(df[df[root]==child][target].unique())==1):
            label = df[df[root]==child][target].value_counts().idxmax()
            tree[root][child] = label
            

        else:
            tree[root][child] =ID3_forest(df[df[root]==child],attributes,depth,m_feature)
    
    return tree

def Bootstrap(data,N_samples):
    np.random.seed(123)
    index = np.random.choice(data.shape[0], N_samples, replace=False)
    #index = np.random.randint(0, data.shape[0],  N_samples)
    Samples = data.iloc[index,:] 
    return Samples

def cross_validation_forest(df,per):
    data = df.sample(frac=1)
    percent = int(data.shape[0]*per/100)
    X_train = data.iloc[0:percent,:]
    X_test = data.iloc[percent:-1,:]
    return X_train,X_test


def RandomForest(df,N_samples,m_feature,num_tree):
    
    #split data
    X_train, X_test = cross_validation_forest(df,80) 
    
    #for keeping trees
    forest = []
    for i in range(num_tree):
        Samples = Bootstrap(X_train,N_samples)
        #depth of tree
        depth = 0
        #get all attributes of data
        attributes = df.columns.tolist()
        #remove target attribute
        attributes.remove('Recidivism - Return to Prison numeric')
        #call decision tree!
        result = ID3_forest(Samples,attributes, depth, m_feature)
        forest.append(result)
    
    return forest, X_test


def dfs_forest_classify(id3,node,df,columns,X_test,column):
    if(type(id3[node]) == dict):
        for key in id3[node].keys():
            data = df.copy()
            if node in columns:
                data = data[data[node] == key]
            dfs_forest_classify(id3[node],key,data,columns,X_test,column)
    else:
        X_test.loc[df.index.tolist(),column] = id3[node]
        return
    return X_test

def prediction_forest(X_Test,forest,num_tree):
    for i in range(num_tree):
        column = f'Y_predict_{i}'
        X_Test[column] = -1
        columns = X_Test.columns.tolist()
        X_Test = dfs_forest_classify(forest[i],list(forest[i].keys())[0],X_Test,columns,X_Test,column)
        
    X_Test['Y_predict'] = X_Test.iloc[:,11:-1].sum(axis=1)
    X_Test['Y_predict'] = np.where(X_Test['Y_predict'] > num_tree/2, 1, 0)
    X_Test = X_Test.drop(X_Test.columns[11:-1], axis=1)
    return X_Test

def main_forest(df,N_samples,m_feature,num_tree):
    forest, X_test = RandomForest(df,N_samples,m_feature,num_tree)
    X_test = prediction_forest(X_test,forest,num_tree)
    return forest,X_test  

In [115]:
data_train = pd.read_csv('prison_dataset.csv')
forest,X_test_f = main_forest(data_train,9000,3,15)
X_test_f

,Fiscal Year Released,Recidivism Reporting Year,Race - Ethnicity,Age At Release,Convicting Offense Classification,Convicting Offense Type,Convicting Offense Subtype,Main Supervising District,Release Type,Part of Target Population,Recidivism - Return to Prison numeric,Y_predict
10594,2015,2018,Black,<45,Other Felony,Other,Other,3JD,Discharged End of Sentence,No,0,0
2493,2013,2016,White,>45,Other Felony,Other,Other,3JD,Discharged End of Sentence,No,1,1
13319,2015,2018,White,<45,Other Felony,Drug,Trafficking,5JD,Parole,Yes,0,0
4001,2013,2016,White,<45,D Felony,Other,Other,5JD,Parole,Yes,1,1
7539,2015,2018,White,>45,D Felony,Other,Other,3JD,Parole,No,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
13906,2015,2018,White,>45,D Felony,Violent,Other,5JD,Parole,Yes,0,0
6641,2015,2018,White,<45,Other Felony,Other,Other,5JD,Parole,Yes,1,0
8263,2015,2018,White,<45,Other Felony,Drug,Trafficking,3JD,Parole,No,0,0
13898,2015,2018,White,>45,Other Felony,Drug,Trafficking,5JD,Parole,No,0,0


# Results for Random Forest:

In [116]:
Cm1, Cm2 = Confusion_Matrix(X_test_f)  
display(Cm1)
display(Cm2)

Predicted:         
                   Positive Negative
Actual: Positive       1099      205
        Negative        628     1152

,Accuracy,Precision,Recall,F1_Score
0,72.989624,63.636364,84.279141,72.517321


# c. RandomForest by ‫‪Scikit-Learn‬‬:

In [102]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

X_train, X_test, y_train, y_test = train_test_split\
(data_train.iloc[:,0:10], data_train.iloc[:,-1], test_size=0.2)
columns = X_train.columns.tolist()
for col in columns:
    if X_train.dtypes[col] == np.object:
        X_train[col] = LabelEncoder().fit_transform(X_train[col])
        X_test[col] = LabelEncoder().fit_transform(X_test[col])
        
rf = RandomForestClassifier(max_depth=3, random_state=0)
rf.fit(X_train, y_train)
predicted = rf.predict(X_test)

df = pd.DataFrame(data={'y_true':y_test,'y_pred':predicted})
Cm1, Cm2 = Confusion_Matrix(df )  
display(Cm1)
display(Cm2)

Predicted:         
                   Positive Negative
Actual: Positive       1120      231
        Negative        624     1110

,Accuracy,Precision,Recall,F1_Score
0,72.285251,64.220183,82.901554,72.374798
